In [ ]:
import logging
import os
import re
import time
import unicodedata
from datetime import datetime
from urllib.parse import urljoin

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# --- Configuración ---
BASE_URL = "https://www.vatican.va/"
OUTPUT_DIR = "archivo_papal"
LOG_FILE = "archiver.log"
REQUEST_DELAY_SECONDS = 1
REQUEST_TIMEOUT_SECONDS = 30  # Timeout para conexión y lectura

# Mapeo de nombres de Papas a sus slugs en la URL
POPE_MAP = {
    "León XIV": "leo-xiv",
    # "Francisco": "francesco",
    # "Benedicto XVI": "benedict-xvi",
    # "Juan Pablo II": "john-paul-ii",
    # "Juan Pablo I": "john-paul-i",
    # "Pablo VI": "paul-vi",
    # "Juan XXIII": "john-xxiii",
    # "Pío XII": "pius-xii",
    # "Pío XI": "pius-xi",
    # "Benedicto XV": "benedict-xv",
    # "Pío X": "pius-x",
    # "León XIII": "leo-xiii",
}

# Configuración del logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)


# --- Funciones de Red y Extracción ---

def get_soup(url):
    """
    Realiza una solicitud GET a una URL de forma robusta,
    maneja errores y devuelve un objeto BeautifulSoup.
    """
    try:
        time.sleep(REQUEST_DELAY_SECONDS)
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        # Usar el timeout definido en la configuración
        response = requests.get(
            url, headers=headers, timeout=REQUEST_TIMEOUT_SECONDS
        )
        response.raise_for_status()
        return BeautifulSoup(response.content, "html.parser")
    except requests.exceptions.RequestException as e:
        logging.error(f"Error al acceder a la URL {url}: {e}")
        return None


def get_pope_main_page_url(pope_slug):
    """Construye la URL de la página principal de un Papa."""
    return urljoin(BASE_URL, f"content/{pope_slug}/es.html")


def extraer_fecha_desde_url(url):
    """Extrae y formatea la fecha desde la URL de un documento."""
    match = re.search(r"/(\d{8})-", url)
    if match:
        fecha_str = match.group(1)
        try:
            fecha_dt = datetime.strptime(fecha_str, "%Y%m%d")
            return fecha_dt.strftime("%d de %B de %Y")
        except ValueError:
            return ""
    return ""


def limpiar_nombre_archivo(nombre):
    """Limpia y normaliza una cadena para usarla como nombre de archivo."""
    nombre = (
        unicodedata.normalize("NFKD", nombre)
        .encode("ascii", "ignore")
        .decode("ascii")
    )
    nombre = re.sub(r"[^\w\s-]", "", nombre)
    nombre = re.sub(r"[-\s]+", "_", nombre)
    return nombre.strip("_").lower()[:100]


def obtener_homilias_vaticano(url):
    """
    Obtiene la lista de URLs de homilías individuales desde una página de índice.
    """
    soup = get_soup(url)
    if not soup:
        logging.error(f"Error al obtener la página de índice: {url}")
        return []

    homilias = []
    contenedor = soup.find("div", class_="vaticanindex")
    if not contenedor:
        logging.warning(f"No se encontró 'vaticanindex' en {url}.")
        return []

    for li in contenedor.find_all("li"):
        enlace = li.find("a")
        if enlace and enlace.get("href"):
            titulo = enlace.get_text(strip=True)
            href = urljoin(BASE_URL, enlace["href"])
            homilias.append({"titulo": titulo, "url": href})

    return homilias


def extraer_homilia(homilia):
    """
    Extrae el contenido de una URL de homilía individual.
    """
    url = homilia["url"]
    titulo_homilia = homilia["titulo"]

    soup = get_soup(url)
    if not soup:
        logging.warning(f"No se pudo obtener el soup para la homilía: {url}. Saltando.")
        return None

    div_testo = soup.find("div", class_="testo")
    if not div_testo:
        logging.warning(f"No se encontró 'div_testo' en {url}. Saltando.")
        return None

    texto_container = div_testo.find("div", class_="text")
    if not texto_container:
        logging.warning(f"No se encontró 'div.text' en {url}. Saltando.")
        return None
        
    texto = "\n".join(
        p.get_text(separator=" ", strip=True)
        for p in texto_container.find_all("p")
    )
    fecha = extraer_fecha_desde_url(url)

    return {
        "url": url,
        "fecha": fecha,
        "titulo_homilia": titulo_homilia,
        "texto": texto,
    }


def process_pope(pope_slug):
    """
    Recopila las URLs de las páginas de índice para un Papa específico.
    """
    pope_main_page_url = get_pope_main_page_url(pope_slug)
    soup = get_soup(pope_main_page_url)
    if not soup:
        logging.error(f"No se pudo acceder a la página principal de {pope_slug}. Saltando.")
        return {}

    urls_by_type = {}
    content_area = soup.find("div", class_="document-container") or soup.body

    for link in content_area.find_all("a", href=True):
        href = link["href"]
        if pope_slug in href and ".html" in href and not href.startswith(("http", "#")):
            full_url = urljoin(pope_main_page_url, href)
            href_lower = href.lower()
            
            tipo = "Otros" # Default
            if "apost_letters" in href_lower: tipo = "Carta Apostólica"
            elif "homilies" in href_lower: tipo = "Homilía"
            elif "speeches" in href_lower: tipo = "Discurso"
            elif "letters" in href_lower: tipo = "Carta"
            elif "messages" in href_lower: tipo = "Mensaje"
            elif "angelus" in href_lower: tipo = "Angelus"
            elif "audiences" in href_lower: tipo = "Audiencia"
            elif "encyclicals" in href_lower: tipo = "Encíclica"
            elif "cotidie" in href_lower: tipo = "Meditaciones Diarias"
            elif "travels" in href_lower: tipo = "Viaje"
            
            urls_by_type.setdefault(tipo, set()).add(full_url)

    return urls_by_type


def obtener_todos_los_textos(urls_por_tipo):
    """
    Orquesta el proceso de scraping para todas las URLs y devuelve un DataFrame.
    """
    all_homilias = []

    for tipo, lista_urls in tqdm(urls_por_tipo.items(), desc="Procesando tipos"):
        for url in tqdm(lista_urls, desc=f"Scraping {tipo}", leave=False):
            homilias_en_pagina = obtener_homilias_vaticano(url)
            
            resultados = [extraer_homilia(h) for h in homilias_en_pagina]
            homilias_extraidas = [h for h in resultados if h is not None]

            for h in homilias_extraidas:
                h["tipo"] = tipo
            all_homilias.extend(homilias_extraidas)

    if not all_homilias:
        logging.warning("No se extrajo ningún documento.")
        return pd.DataFrame()

    df = pd.DataFrame(all_homilias)
    df = df.rename(columns={"titulo_homilia": "titulo"})
    df = df[["tipo", "fecha", "titulo", "url", "texto"]]
    
    # Intentar establecer el locale para el parseo de fechas en español
    try:
        import locale
        locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
    except locale.Error:
        logging.warning("Locale 'es_ES.UTF-8' no disponible. Los nombres de los meses podrían no parsearse correctamente.")

    df["fecha"] = pd.to_datetime(df["fecha"], format="%d de %B de %Y", errors="coerce")
    df = df.dropna(subset=['fecha'])
    df = df.sort_values(by="fecha", ascending=False).reset_index(drop=True)
    df["fecha"] = df["fecha"].dt.strftime("%Y-%m-%d")

    return df


# --- Función Principal ---

def main():
    """
    Función principal que ejecuta el proceso de archivado.
    """
    logging.info("Iniciando el Archivador Papal.")
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        logging.info(f"Directorio de salida creado: {OUTPUT_DIR}")

    final_df = pd.DataFrame()

    for pope_name, pope_slug in POPE_MAP.items():
        logging.info(f"--- Iniciando procesamiento para el Papa: {pope_name} ---")
        urls_a_scrapear = process_pope(pope_slug)
        
        if not urls_a_scrapear:
            logging.warning(f"No se encontraron URLs para {pope_name}. Continuando.")
            continue
            
        logging.info(f"Encontradas {sum(len(v) for v in urls_a_scrapear.values())} páginas de índice para {pope_name}.")
        
        df_papa = obtener_todos_los_textos(urls_a_scrapear)
        
        if not df_papa.empty:
            df_papa['papa'] = pope_name
            final_df = pd.concat([final_df, df_papa], ignore_index=True)

    if final_df.empty:
        logging.info("No se generaron datos. Proceso finalizado.")
        return

    # Limpieza final y generación de nombres de archivo
    final_df = final_df[final_df["titulo"].str.len() > 10].reset_index(drop=True)
    final_df["filename"] = final_df.apply(
        lambda row: f"{row['fecha']}_{limpiar_nombre_archivo(row['tipo'])}_{limpiar_nombre_archivo(row['titulo'])}.md",
        axis=1,
    )
    
    # Guardar el DataFrame completo en un archivo CSV
    output_csv_path = os.path.join(OUTPUT_DIR, "magisterio_completo.csv")
    final_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    logging.info(f"DataFrame completo guardado en: {output_csv_path}")

    # --- Guardar cada texto como un archivo Markdown individual ---
    logging.info("Guardando documentos individuales como archivos Markdown...")

    for index, row in tqdm(final_df.iterrows(), total=final_df.shape[0], desc="Guardando archivos MD"):
        # Crear la ruta de directorio estructurada: output_dir/nombre_del_papa/tipo_de_documento/
        pope_slug = limpiar_nombre_archivo(row['papa'])
        type_slug = limpiar_nombre_archivo(row['tipo'])
        
        dir_path = os.path.join(OUTPUT_DIR, pope_slug, type_slug)
        os.makedirs(dir_path, exist_ok=True) # Crea la estructura de carpetas si no existe
        
        # Ruta completa del archivo
        md_path = os.path.join(dir_path, row['filename'])
        
        # Escribir el contenido en el archivo Markdown
        try:
            with open(md_path, 'w', encoding='utf-8') as f:
                f.write(f"# {row['titulo']}\n\n")
                f.write(f"**Papa:** {row['papa']}\n")
                f.write(f"**Tipo:** {row['tipo']}\n")
                f.write(f"**Fecha:** {row['fecha']}\n")
                # Se añade la URL como un enlace clicable en Markdown
                f.write(f"**URL Original:** [{row['url']}]({row['url']})\n\n")
                f.write("---\n\n")
                f.write(row['texto'] if pd.notna(row['texto']) else "")
        except IOError as e:
            logging.error(f"No se pudo escribir el archivo {md_path}: {e}")

    logging.info("Proceso de archivado completado.")
    print("\n--- Muestra del DataFrame Final ---")
    return final_df


final_df =main()

2025-08-09 02:17:06,820 - INFO - Iniciando el Archivador Papal.
2025-08-09 02:17:06,824 - INFO - --- Iniciando procesamiento para el Papa: León XIV ---


2025-08-09 02:17:08,144 - INFO - Encontradas 24 páginas de índice para León XIV.
Procesando tipos: 100%|██████████| 7/7 [03:08<00:00, 26.86s/it]
2025-08-09 02:20:16,210 - INFO - No se generaron datos. Proceso finalizado.


In [ ]:
urls

KeyError: 0

  9%|▉         | 1/11 [02:52<28:48, 172.84s/it]


ConnectTimeout: HTTPSConnectionPool(host='www.vatican.va', port=443): Max retries exceeded with url: /content/francesco/es/angelus/2021/documents/papa-francesco_angelus_20210530.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1236d0860>, 'Connection to www.vatican.va timed out. (connect timeout=None)'))

In [4]:
df

,tipo,fecha,titulo,url,texto,filename
0,Audiencia,2025-08-06,Audiencia general del 6 de agosto de 2025 Cicl...,https://www.vatican.va/content/leo-xiv/es/audi...,LEÓN XIV\nAUDIENCIA GENERAL\nPlaza de San Pedr...,2025-08-06_audiencia_audiencia_general_del_6_d...
1,Mensaje,2025-08-06,Videomensaje del Santo Padre a la Red Panafric...,https://www.vatican.va/content/leo-xiv/es/mess...,VIDEOMENSAJE DE SU SANTIDAD EL PAPA LEÓN XIV A...,2025-08-06_mensaje_videomensaje_del_santo_padr...
2,Homilía,2025-08-03,Santa Misa por el Jubileo de los Jóvenes (3 de...,https://www.vatican.va/content/leo-xiv/es/homi...,JUBILEO DE LOS JÓVENES\nSANTA MISA\nHOMILÍA DE...,2025-08-03_homilía_santa_misa_por_el_jubileo_d...
3,Angelus,2025-08-03,"Ángelus, 3 de agosto de 2025",https://www.vatican.va/content/leo-xiv/es/ange...,"PAPA LEÓN XIV\nÁNGELUS\nTor Vergata Domingo, 3...",2025-08-03_angelus_angelus_3_de_agosto_de_2025
4,Discurso,2025-08-02,Vigilia de oración con los jóvenes (Tor Vergat...,https://www.vatican.va/content/leo-xiv/es/spee...,VIGILIA CON LOS JÓVENES\nDIÁLOGO DEL SANTO PAD...,2025-08-02_discurso_vigilia_de_oracion_con_los...
...,...,...,...,...,...,...
94,Angelus,2025-05-11,"Regina Caeli, 11 de mayo de 2025",https://www.vatican.va/content/leo-xiv/es/ange...,PAPA LEÓN XIV\nREGINA CAELI\nLogia Central de ...,2025-05-11_angelus_regina_caeli_11_de_mayo_de_...
95,Homilía,2025-05-11,Homilía del Santo Padre León XIV en la cripta ...,https://www.vatican.va/content/leo-xiv/es/homi...,HOMILÍA DEL SANTO PADRE LEÓN XIV EN LA CRIPTA ...,2025-05-11_homilía_homilia_del_santo_padre_leo...
96,Discurso,2025-05-10,Discurso del Santo Padre al Colegio Cardenalic...,https://www.vatican.va/content/leo-xiv/es/spee...,DISCURSO DEL SANTO PADRE LEÓN XIV AL COLEGIO C...,2025-05-10_discurso_discurso_del_santo_padre_a...
97,Homilía,2025-05-09,Santa Misa pro Ecclesia celebrada por el Roman...,https://www.vatican.va/content/leo-xiv/es/homi...,SANTA MISA PRO ECCLESIA CELEBRADA POR EL ROMAN...,2025-05-09_homilía_santa_misa_pro_ecclesia_cel...
